In [33]:
from crawling import *
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
def count_progress(cnt, data, division):
    if cnt // (len(data) // division) != (cnt-1) // (len(data) // division):
        print((cnt // (len(data) // division))*100/division, "%")

In [14]:
# first process
root_url = "https://comic.naver.com/webtoon/weekday"
first_data = {}
for tag in html_parser(root_url, dynamic=False).select(".thumb>a"):
    title = tag.select_one("img").attrs["title"]
    url = "https://comic.naver.com" + tag.attrs["href"]
    weekday = url.split("weekday=")[-1]
    if title in first_data:
        first_data[title]["weekday"] = first_data[title]["weekday"] + [weekday]
    else:
        first_data[title] =  {"url":url, "weekday":[weekday]}
print(first_data)

{'쇼미더럭키짱!': {'url': 'https://comic.naver.com/webtoon/list?titleId=783054&weekday=mon', 'weekday': ['mon', 'tue', 'wed', 'thu', 'fri']}, '참교육': {'url': 'https://comic.naver.com/webtoon/list?titleId=758037&weekday=mon', 'weekday': ['mon']}, '2022 서브병에 빠지다!': {'url': 'https://comic.naver.com/webtoon/list?titleId=787411&weekday=mon', 'weekday': ['mon', 'tue', 'fri', 'sat']}, '신의 탑': {'url': 'https://comic.naver.com/webtoon/list?titleId=183559&weekday=mon', 'weekday': ['mon']}, '퀘스트지상주의': {'url': 'https://comic.naver.com/webtoon/list?titleId=783052&weekday=mon', 'weekday': ['mon']}, '소녀의 세계': {'url': 'https://comic.naver.com/webtoon/list?titleId=654774&weekday=mon', 'weekday': ['mon']}, '장씨세가 호위무사': {'url': 'https://comic.naver.com/webtoon/list?titleId=728750&weekday=mon', 'weekday': ['mon']}, '백수세끼': {'url': 'https://comic.naver.com/webtoon/list?titleId=733074&weekday=mon', 'weekday': ['mon']}, '팔이피플': {'url': 'https://comic.naver.com/webtoon/list?titleId=774863&weekday=mon', 'weekday': ['

In [7]:
import sys
# datetime has smaller size than string!
print(sys.getsizeof(datetime(2021,8,21)), sys.getsizeof("2021.08.21"))

48 59


In [25]:
# second process
second_data = {}
cnt = 0
for title in first_data:
    count_progress(cnt, first_data, 10)
    cnt += 1
    html = html_parser(first_data[title]["url"], dynamic=False)
    title_info = {}
    title_info["url"] = first_data[title]["url"]
    title_info["writer"] = get_text(html.select_one(".wrt_nm")).split("/")
    title_info["genre"] = get_text(html.select_one(".genre")).split(",")
    title_info["weekday"] = first_data[title]["weekday"]
    title_info["age"] = get_text(html.select_one(".age"))
    rating_info = {}
    rating, date, target_url =  [], [], first_data[title]["url"]
    while True:
        target_html = html_parser(target_url, dynamic=False)
        temp_rating = list(map(float, get_text(target_html.select(".rating_type>strong"))))
        temp_date = list(map(lambda x:datetime.strptime(x, "%Y.%m.%d"), get_text(target_html.select(".num"))))
        for i in range(len(temp_date)):
            if temp_date[i].year == 2020:
                break
            date.append(temp_date[i])
            rating.append(temp_rating[i])
        else:
            next = target_html.select_one(".next")
            if next is not None:
                target_url = "https://comic.naver.com" + next.attrs["href"]
                continue
        break
    title_info["rating_info"] = {"rating":rating, "date":date}
    second_data[title] = title_info
print(len(second_data))

0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %
493


In [26]:
print(second_data)

{'쇼미더럭키짱!': {'url': 'https://comic.naver.com/webtoon/list?titleId=783054&weekday=mon', 'writer': ['박태준 ', ' 김성모'], 'genre': ['에피소드', ' 액션'], 'weekday': ['mon', 'tue', 'wed', 'thu', 'fri'], 'age': '15세 이용가', 'rating_info': {'rating': [9.74, 9.77, 9.77, 9.78, 9.74, 9.72, 9.74, 9.78, 9.77, 9.74, 9.74, 9.73, 9.74, 9.76, 9.74, 9.75, 9.74, 9.77, 9.86, 9.69, 9.69, 9.77, 9.69, 9.7, 9.63, 9.7, 9.67, 9.59, 9.62, 9.62, 9.61, 9.59, 9.64, 9.64, 9.66, 9.66, 9.6, 9.51, 9.6, 9.51, 9.57, 9.58, 9.4, 9.42, 9.26, 9.23, 9.21, 9.04, 8.89, 8.52, 8.16, 8.17, 7.98, 6.88], 'date': [datetime.datetime(2022, 1, 26, 0, 0), datetime.datetime(2022, 1, 25, 0, 0), datetime.datetime(2022, 1, 24, 0, 0), datetime.datetime(2022, 1, 23, 0, 0), datetime.datetime(2022, 1, 20, 0, 0), datetime.datetime(2022, 1, 19, 0, 0), datetime.datetime(2022, 1, 18, 0, 0), datetime.datetime(2022, 1, 17, 0, 0), datetime.datetime(2022, 1, 16, 0, 0), datetime.datetime(2022, 1, 13, 0, 0), datetime.datetime(2022, 1, 12, 0, 0), datetime.datetime(2

In [31]:
# preprocess
# removing title that has not enough information
pre_data = {}
for title in second_data:
    if len(second_data[title]["rating_info"]["rating"])/len(second_data[title]["weekday"]) >= 40: # total serialized week count of webtoon
        pre_data[title] = second_data[title]
print(len(pre_data))

233


In [32]:
pre_data

{'참교육': {'url': 'https://comic.naver.com/webtoon/list?titleId=758037&weekday=mon',
  'writer': ['채용택 ', ' 한가람'],
  'genre': ['스토리', ' 액션'],
  'weekday': ['mon'],
  'age': '15세 이용가',
  'rating_info': {'rating': [9.94,
    9.95,
    9.83,
    9.91,
    9.93,
    9.91,
    9.93,
    9.93,
    9.96,
    9.96,
    9.94,
    9.44,
    9.91,
    9.91,
    9.94,
    9.94,
    9.95,
    9.93,
    9.87,
    9.8,
    9.83,
    9.7,
    9.22,
    9.6,
    9.86,
    9.88,
    9.9,
    9.9,
    9.9,
    9.91,
    9.93,
    9.92,
    9.93,
    9.92,
    9.9,
    9.9,
    9.92,
    9.9,
    9.92,
    9.92,
    9.9,
    9.91,
    9.36,
    9.74,
    9.59,
    9.92,
    9.94,
    9.93,
    9.91,
    9.83,
    9.91,
    9.92,
    9.89,
    9.8,
    9.72],
   'date': [datetime.datetime(2022, 1, 23, 0, 0),
    datetime.datetime(2022, 1, 16, 0, 0),
    datetime.datetime(2022, 1, 9, 0, 0),
    datetime.datetime(2022, 1, 2, 0, 0),
    datetime.datetime(2021, 12, 26, 0, 0),
    datetime.datetime(2021, 12, 19, 

In [48]:
temp = []
max_rating_urls = []
min_rating_urls = []
for title in pre_data:
    temp.append((np.mean(pre_data[title]["rating_info"]["rating"]), title))
temp = sorted(temp)
for _,title in temp[:5]:
    min_rating_urls.append((title, pre_data[title]["url"]))
for _,title in temp[-1:-6:-1]:
    max_rating_urls.append((title,pre_data[title]["url"]))
print("max_rating_urls")
print(max_rating_urls)
print("min_rating_urls")
print(min_rating_urls)

max_rating_urls
[('세기말 풋사과 보습학원', 'https://comic.naver.com/webtoon/list?titleId=761722&weekday=fri'), ('마른 가지에 바람처럼', 'https://comic.naver.com/webtoon/list?titleId=748535&weekday=wed'), ('숲속의 담', 'https://comic.naver.com/webtoon/list?titleId=738145&weekday=tue'), ('엔딩 후 서브남을 주웠다', 'https://comic.naver.com/webtoon/list?titleId=761498&weekday=wed'), ('야생천사 보호구역', 'https://comic.naver.com/webtoon/list?titleId=767919&weekday=mon')]
min_rating_urls
[('달의 요람', 'https://comic.naver.com/webtoon/list?titleId=762035&weekday=thu'), ('엽사:요괴사냥꾼', 'https://comic.naver.com/webtoon/list?titleId=769210&weekday=fri'), ('헬퍼 2 : 킬베로스', 'https://comic.naver.com/webtoon/list?titleId=670143&weekday=wed'), ('더 게이머', 'https://comic.naver.com/webtoon/list?titleId=552960&weekday=fri'), ('여신강림', 'https://comic.naver.com/webtoon/list?titleId=703846&weekday=tue')]


In [2]:
temp = []
max_var_rating_urls = []
for title in pre_data:
    temp.append((np.var(pre_data[title]["rating_info"]["rating"]), title))
temp = sorted(temp)
for _,title in temp[-1:-11:-1]:
    max_var_rating_urls.append((title,pre_data[title]["url"]))
print(max_var_rating_urls)

NameError: name 'pre_data' is not defined

In [1]:
target_title = max_var_rating_urls[0][0]
plt.plot(pre_data[target_title]["rating_info"]["date"],pre_data[target_title]["rating_info"]["rating"])

NameError: name 'max_var_rating_urls' is not defined